In [11]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime

# Global Model

In [12]:
no_of_noaa_records = 7431587

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'noaa_nam_2'
pipeline = [
    {
        "$sample": {"size": 74315}
    }
]
cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [13]:
def format_date(t):
    t = str(t)
    date = datetime.strptime(t, '%Y%m%d%H')
    return f'{date.year}-{date.month}-{date.day} {str(date.hour).zfill(2)}:{str(date.minute).zfill(2)}:{str(date.second).zfill(2)}'


df = df_original
df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
df

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909cf9cbf00adffbffebbd,2010-1-18 06:00:00,0,G3000810,45.980309,-113.756459,100499.0,78532.0,2020.725159,271.748016,...,3.792786,0.0,0.0,271.886841,22167.625427,24232.568449,8.347592,100.0,0.92384,0.0
1,60909b419b1064bc9b65f4d4,2010-1-14 06:00:00,0,G3101810,40.005045,-98.377505,101614.0,94838.0,551.475159,267.081070,...,-3.002218,0.0,0.0,269.462662,24160.507202,24231.905305,7.476454,50.0,0.13040,0.0
2,60909b47184c9c5cb32de4cc,2010-1-14 06:00:00,3,G4805030,33.121854,-98.948407,101687.0,97176.0,377.725159,279.688293,...,5.086803,0.0,0.0,280.472977,26160.438538,24233.232546,13.818760,2.0,0.00000,0.0
3,60909ac6eb59fe9695330c61,2010-1-13 00:00:00,3,G4100490,45.595216,-119.896484,101295.0,97403.0,324.225159,276.188812,...,1.612885,0.0,0.0,278.496796,30360.496521,24231.373781,15.536448,0.0,0.00000,0.0
4,609099f60394db46bd130a9c,2010-1-11 00:00:00,6,G2701190,47.676171,-96.860614,102833.0,99391.0,261.725159,262.957962,...,-3.954071,0.0,0.0,265.190552,22160.479736,24227.428675,8.429266,100.0,0.29440,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74310,60909b5bccef3c16a9f793ea,2010-1-14 12:00:00,0,G3500550,36.720169,-105.649923,101741.0,75265.0,2467.225159,271.104492,...,-1.448517,0.0,0.0,271.223434,26160.507202,24035.177785,5.828953,100.0,0.02600,0.0
74311,60909c59cdc0508afddb364b,2010-1-16 18:00:00,3,G3500450,36.688852,-107.662261,102016.0,80939.0,1893.725159,274.554642,...,-0.519119,0.0,0.0,271.150681,20360.455322,24039.371485,4.985330,6.0,0.03176,0.0
74312,60909c7e391e36c462f8e84c,2010-1-17 00:00:00,6,G1700250,38.749358,-88.509283,101384.0,99526.0,148.225159,273.630035,...,-2.107666,0.0,0.0,273.393509,22167.608643,39.046925,17.516079,100.0,0.00552,0.0
74313,60909bd2ba60520913262a6a,2010-1-15 12:00:00,3,G3100850,40.458684,-101.168363,102647.0,91609.0,918.725159,273.678894,...,-1.958503,0.0,0.0,272.623764,24467.372131,24035.608697,8.970860,96.0,0.00000,0.0


In [53]:
noaa_features = [
    "mean_sea_level_pressure_pascal",
    "surface_pressure_surface_level_pascal",
    "orography_surface_level_meters",
    "temp_surface_level_kelvin",
    "2_metre_temp_kelvin",
    "2_metre_dewpoint_temp_kelvin",
    "relative_humidity_percent",
    "10_metre_u_wind_component_meters_per_second",
    "10_metre_v_wind_component_meters_per_second",
    "total_precipitation_kg_per_squared_meter",
    "water_convection_precipitation_kg_per_squared_meter",
    "soil_temperature_kelvin",
    "pressure_pascal",
    "visibility_meters",
    "precipitable_water_kg_per_squared_meter",
    "total_cloud_cover_percent",
    "snow_depth_meters",
    "ice_cover_binary"
]

unique_timestamps = df['year_month_day_hour'].unique()
df_map = {}

for selected_feature in noaa_features:
    df_s = df[['gis_join', 'year_month_day_hour', selected_feature]]
    means = []
    for t in unique_timestamps:
        mean = df_s[df_s['year_month_day_hour'] == t][selected_feature].mean()
        means.append(mean)
    df_means = pd.DataFrame(list(zip(list(df_s['gis_join']), unique_timestamps, means)), columns=['gis_join', 'ds', 'y'])
#     df_means.index.name = 'ds'
    df_map[selected_feature] = df_means
    
df_map['mean_sea_level_pressure_pascal'].head()

,gis_join,ds,y
0,G3000810,2010-1-18 06:00:00,101271.974725
1,G3101810,2010-1-14 06:00:00,102106.333492
2,G4805030,2010-1-13 00:00:00,102206.214195
3,G4100490,2010-1-11 00:00:00,102702.483703
4,G2701190,2010-1-19 12:00:00,100975.370331


In [27]:
pickle.dump(df_map, open('pickles/noaa/df_map.pkl', 'wb'))

In [28]:
# Start by loading pickled 'df_map'
df_map = pickle.load(open('pickles/noaa/df_map.pkl', 'rb'))

In [54]:
# select one feature
df0 = df_map['mean_sea_level_pressure_pascal']
print(df0.columns)
df0.head()

Index(['gis_join', 'ds', 'y'], dtype='object')


,gis_join,ds,y
0,G3000810,2010-1-18 06:00:00,101271.974725
1,G3101810,2010-1-14 06:00:00,102106.333492
2,G4805030,2010-1-13 00:00:00,102206.214195
3,G4100490,2010-1-11 00:00:00,102702.483703
4,G2701190,2010-1-19 12:00:00,100975.370331


## Build Global Model

In [55]:
def predict(df_train):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return fit_m, df_train_future, df_train_forecast

fit_m, df_train, df_train_forecast = predict(df0)

Initial log joint probability = -2.02017
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       327.687    0.00163465       2735.57           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       331.593   7.39265e-05       2032.74           1           1      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       336.412   0.000115985       609.251           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       338.096    0.00149963       4737.92           1           1      469   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       338.766   5.78605e-06       1328.15           1           1      583   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [59]:
df_train_forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2010-01-10 00:00:00,101551.064379,102948.693931,103016.681156,101551.064379,101551.064379,1432.010368,1432.010368,1432.010368,-712.400035,...,-51.037372,-51.037372,-51.037372,2195.447774,2195.447774,2195.447774,0.0,0.0,0.0,102983.074747
1,2010-01-10 06:00:00,101508.464441,102969.015830,103042.135828,101508.464441,101508.464441,1496.989724,1496.989724,1496.989724,-655.171779,...,-7.225910,-7.225910,-7.225910,2159.387412,2159.387412,2159.387412,0.0,0.0,0.0,103005.454165
2,2010-01-10 12:00:00,101444.586884,102927.461297,102996.967545,101444.586884,101444.586884,1515.430874,1515.430874,1515.430874,-645.638086,...,45.430238,45.430238,45.430238,2115.638722,2115.638722,2115.638722,0.0,0.0,0.0,102960.017758
3,2010-01-10 18:00:00,101380.709326,102666.506646,102736.155931,101380.709326,101380.709326,1322.218315,1322.218315,1322.218315,-831.925676,...,89.844865,89.844865,89.844865,2064.299126,2064.299126,2064.299126,0.0,0.0,0.0,102702.927642
4,2010-01-11 00:00:00,101316.059741,102689.599335,102756.872257,101316.059741,101316.059741,1408.257029,1408.257029,1408.257029,-712.400035,...,115.177830,115.177830,115.177830,2005.479234,2005.479234,2005.479234,0.0,0.0,0.0,102724.316770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,2010-02-02 02:00:00,112248.604800,99413.932280,117517.812069,102712.312426,120802.651184,-3296.112087,-3296.112087,-3296.112087,335.225622,...,41.671860,41.671860,41.671860,-3673.009569,-3673.009569,-3673.009569,0.0,0.0,0.0,108952.492713
340,2010-02-02 03:00:00,112272.066039,99795.950504,117949.283917,102692.009340,120838.639088,-2896.067856,-2896.067856,-2896.067856,711.283856,...,36.012749,36.012749,36.012749,-3643.364460,-3643.364460,-3643.364460,0.0,0.0,0.0,109375.998183
341,2010-02-02 04:00:00,112295.527277,99477.670701,117694.060715,102671.706255,120874.388133,-3197.273693,-3197.273693,-3197.273693,386.123173,...,30.181132,30.181132,30.181132,-3613.577998,-3613.577998,-3613.577998,0.0,0.0,0.0,109098.253585
342,2010-02-02 05:00:00,112318.988516,98777.243174,117068.120901,102630.089336,120940.849398,-3861.663692,-3861.663692,-3861.663692,-302.173136,...,24.160840,24.160840,24.160840,-3583.651396,-3583.651396,-3583.651396,0.0,0.0,0.0,108457.324824


In [57]:
global_model = fit_m

In [32]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df0_cv = cross_validation(fit_m, initial='2 days', period='1 days', horizon='1 days')
df0_p = performance_metrics(df0_cv)
df0_p.head()

INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

INFO:prophet:n_changepoints greater than number of observations. Using 8.


Initial log joint probability = -2.02001
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       108.249     0.0152024       52168.4           1           1      143   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       117.698   0.000794267       16657.1           1           1      262   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       118.768   0.000251278       3576.35           1           1      373   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       119.558   0.000257061       6617.66      0.1154           1      497   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       119.882   0.000518041       3862.85           1           1      610   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:n_changepoints greater than number of observations. Using 11.


...

Initial log joint probability = -2.02008
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       215.713    0.00292013       4946.44           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       223.793     0.0123567         32023           1           1      245   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       227.797   0.000554952       809.105           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       228.407   6.72774e-05       499.469           1           1      469   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       232.778   0.000385214        1970.8           1           1      582   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:n_changepoints greater than number of observations. Using 24.


Initial log joint probability = -2.02018
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       249.369      0.022786       33941.9           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       254.056   7.58325e-05       1215.34           1           1      234   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       254.671   0.000251582       3213.86           1           1      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       257.666   0.000214433       406.218           1           1      461   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       257.968   0.000401507       1632.18           1           1      573   
    Iter      log prob        ||dx||      ||grad||       alpha  

    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    8382       279.776   1.37791e-07       1916.94   4.186e-11       0.001    12165  LS failed, Hessian reset 
    8399       279.787   9.85641e-07       628.856           1           1    12184   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    8464       279.866   4.84805e-08       725.728   5.154e-11       0.001    12306  LS failed, Hessian reset 
    8499         279.9   3.03976e-05        1568.2      0.8146      0.8146    12345   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
    8598       279.932   5.90214e-08       114.104      0.2396      0.9824    12465   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
Initial log joint probability = -2.02012
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
  

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,0 days 06:00:00,1.328643e+04,115.266788,88.618998,0.000870,0.000661,0.000869,0.500
1,0 days 12:00:00,2.290321e+05,478.572941,293.043283,0.002872,0.000758,0.002867,0.500
2,0 days 18:00:00,9.369945e+05,967.984772,545.634937,0.005345,0.000700,0.005331,0.625
3,1 days 00:00:00,2.817623e+06,1678.577699,952.012648,0.009343,0.002475,0.009327,0.750


In [33]:
model = fit_m
print(f'seasonality_prior_scale: {model.seasonality_prior_scale}')
print(f'changepoint_prior_scale: {model.changepoint_prior_scale}')
print(f'uncertainty_samples: {model.uncertainty_samples}')
print(f'seasonlity_mode: {model.seasonality_mode}')
print(f'interval_width: {model.interval_width}')
print(f'growth: {model.growth}')

seasonality_prior_scale: 10.0
changepoint_prior_scale: 0.05
uncertainty_samples: 1000
seasonlity_mode: additive
interval_width: 0.8
growth: linear


## GridSearch on Global Model

In [34]:
import itertools
from dask.distributed import Client

client = Client('lattice-150:8786')    


def grid_search(df):
    param_grid = {  
        'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
        'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
    }

    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    rmses = []  # Store the RMSEs for each params here

    time1 = time.monotonic()
    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, initial= '2 days', period='1 days', horizon='1 days', parallel="dask")
        df_p = performance_metrics(df_cv, rolling_window=1)
        rmses.append(df_p['rmse'].values[0])

    # Find the best parameters
    tuning_results = pd.DataFrame(all_params)
    tuning_results['rmse'] = rmses

    print(f'Time: {time.monotonic() - time1}')
    
    optimal_params = tuning_results[tuning_results['rmse'] == min(tuning_results['rmse'])]
    rmse = optimal_params['rmse'].iloc[0]
    changepoint_prior_scale = optimal_params['changepoint_prior_scale'].iloc[0]
    seasonality_prior_scale = optimal_params['seasonality_prior_scale'].iloc[0]
    
    return rmse, changepoint_prior_scale, seasonality_prior_scale  

In [147]:
rmse, changepoint_prior_scale, seasonality_prior_scale = grid_search(df0)

# Optimal values
print(f'rmse: {rmse}')
print(f'changepoint_prior_scale: {changepoint_prior_scale}')
print(f'seasonality_prior_scale: {seasonality_prior_scale}')

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = 137.119
Iteration  1. Log joint probability =    125.411. Improved by -11.7085.
Iteration  2. Log joint probability =    161.704. Improved by 36.2934.
Iteration  3. Log joint probability =    182.823. Improved by 21.1191.
Iteration  4. Log joint probability =    215.766. Improved by 32.943.
Iteration  5. Log joint probability =    242.688. Improved by 26.9216.
Iteration  6. Log joint probability =    247.787. Improved by 5.09965.
Iteration  7. Log joint probability =    250.059. Improved by 2.27196.
Iteration  8. Log joint probability =    251.532. Improved by 1.47284.
Iteration  9. Log joint probability =    251.621. Improved by 0.0890902.
Iteration 10. Log joint probability =    251.966. Improved by 0.345213.
Iteration 11. Log joint probability =    252.277. Improved by 0.310613.
Iteration 12. Log joint probability =    252.374. Improved by 0.0974569.
Iteration 13. Log joint probability =    252.489. Improved by 0.115004.
Iteration 14. Log joint proba

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = 118.698
Iteration  1. Log joint probability =    125.411. Improved by 6.71279.
Iteration  2. Log joint probability =    161.564. Improved by 36.1531.
Iteration  3. Log joint probability =    180.417. Improved by 18.8523.
Iteration  4. Log joint probability =    217.254. Improved by 36.8375.
Iteration  5. Log joint probability =    241.509. Improved by 24.2546.
Iteration  6. Log joint probability =    250.741. Improved by 9.23254.
Iteration  7. Log joint probability =    250.966. Improved by 0.225089.
Iteration  8. Log joint probability =    251.001. Improved by 0.0346655.
Iteration  9. Log joint probability =    251.578. Improved by 0.576878.
Iteration 10. Log joint probability =    252.357. Improved by 0.779213.
Iteration 11. Log joint probability =    252.471. Improved by 0.11369.
Iteration 12. Log joint probability =    252.651. Improved by 0.180092.
Iteration 13. Log joint probability =    252.782. Improved by 0.131051.
Iteration 14. Log joint proba

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00


...


Initial log joint probability = -55.0875
Iteration  1. Log joint probability =    125.344. Improved by 180.431.
Iteration  2. Log joint probability =    155.052. Improved by 29.7089.
Iteration  3. Log joint probability =    201.434. Improved by 46.3812.
Iteration  4. Log joint probability =    228.506. Improved by 27.0724.
Iteration  5. Log joint probability =    254.772. Improved by 26.2662.
Iteration  6. Log joint probability =    278.104. Improved by 23.3317.
Iteration  7. Log joint probability =     300.51. Improved by 22.4062.
Iteration  8. Log joint probability =    325.575. Improved by 25.0646.
Iteration  9. Log joint probability =    342.052. Improved by 16.4775.
Iteration 10. Log joint probability =     369.64. Improved by 27.588.
Iteration 11. Log joint probability =    371.412. Improved by 1.77216.
Iteration 12. Log joint probability =     372.05. Improved by 0.637813.
Iteration 13. Log joint probability =    372.322. Improved by 0.271881.
Iteration 14. Log joint probabilit

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00



Initial log joint probability = -73.5082
Iteration  1. Log joint probability =    125.342. Improved by 198.85.
Iteration  2. Log joint probability =    157.593. Improved by 32.2513.
Iteration  3. Log joint probability =    199.479. Improved by 41.8853.
Iteration  4. Log joint probability =    226.623. Improved by 27.1447.
Iteration  5. Log joint probability =    253.265. Improved by 26.642.
Iteration  6. Log joint probability =    290.156. Improved by 36.8902.
Iteration  7. Log joint probability =    296.059. Improved by 5.90313.
Iteration  8. Log joint probability =    347.689. Improved by 51.6306.
Iteration  9. Log joint probability =    368.353. Improved by 20.6641.
Iteration 10. Log joint probability =    372.261. Improved by 3.90716.
Iteration 11. Log joint probability =    372.339. Improved by 0.0789348.
Iteration 12. Log joint probability =     372.34. Improved by 0.000323827.
Iteration 13. Log joint probability =     372.34. Improved by 8.42056e-05.
Iteration 14. Log joint pro

INFO:prophet:Applying in parallel with <Client: 'tcp://129.82.208.160:8786' processes=24 threads=384, memory=1.47 TiB>


Time: 706.3904054430313
rmse: 331.1993775284078
changepoint_prior_scale: 0.5
seasonality_prior_scale: 10.0


In [159]:
gis_joins = df_original['gis_join'].unique()
print(f'#GISJOINs: {len(gis_joins)}')

#GISJOINs: 3049


## Dictionary of GISJOIN --> DF

In [171]:
import dask
selected_feature = 'mean_sea_level_pressure_pascal'
child_dfs = {}

def get_df_by_gis_join(gis_join):
    print(gis_join, end=' ')
    cursor = db.sustaindb[collection].aggregate([{"$match": {"gis_join": gis_join}}])
    df = pd.DataFrame(list(cursor))[['year_month_day_hour', selected_feature]]
    df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
    df.columns = ['ds','y']
    return df

futures = []
for gis_join in gis_joins:
    child_dfs[gis_join] = get_df_by_gis_join(gis_join)

G3900930 G0800870 G5600290 G4100370 G0600070 G1600150 G0801090 G4600930 G0800010 G3000710 G3700410 G1900750 G3800310 G4500070 G3900050 G5100710 G2200350 G0600690 G4601170 G3100210 G4900450 G5500290 G1200670 G4802090 G4800710 G3500060 G0100890 G1700210 G1301850 G5400870 G2500230 G2601390 G3700110 G0501190 G0101270 G0600790 G2600730 G1600430 G1701470 G0101130 G0600270 G4001130 G3800470 G2600330 G0500150 G4200890 G1700370 G1700470 G0500930 G5600070 G2200470 G2601510 G1600490 G3200070 G2700110 G3101150 G0800370 G5400750 G4200150 G5600050 G1600210 G0600990 G1200970 G0600890 G1201230 G3101110 G3200170 G5500710 G0400030 G4200310 G3600090 G1201270 G4200350 G1901870 G4700350 G0400230 G2000710 G2600070 G4802290 G3500390 G0600190 G0600710 G3900350 G5000210 G0800750 G5300210 G5600350 G1600350 G0600450 G5500050 G4600090 G1800870 G0201980 G2001870 G4601210 G1302250 G2700410 G3500590 G3500530 G4804650 G4804430 G0800250 G1900390 G2801590 G4200330 G2001490 G1300710 G2701430 G3500190 G4100250 G5400830 G

G5300330 G4900490 G5401070 G4601090 G5500370 G3900290 G4100490 G4804050 G3101250 G2700130 G0100110 G1901490 G3701130 G0600670 G3701030 G5500110 G0801250 G2200150 G2800490 G3100410 G2100010 G3001110 G2900490 G2200650 G5400510 G2001650 G2001110 G2001290 G0100770 G2001670 G0101070 G3400070 G0600250 G5500070 G0100530 G4701150 G3000230 G2200930 G2000690 G1700670 G3100450 G3000090 G2902030 G4601350 G0800490 G2801010 G2901030 G4803670 G2601450 G4804330 G2200970 G4802250 G3701370 G4400090 G2001450 G2601210 G5101570 G1300690 G1701510 G2600170 G1300530 G0100550 G1900810 G1300950 G4200850 G3600410 G1301410 G3000910 G3901250 G3700510 G5101870 G2200750 G2800350 G4000330 G4701770 G3000830 G5400930 G5500930 G4900530 G4701030 G4803650 G0801130 G4802510 G5600410 G2901230 G5400190 G2900810 G4100010 G0900110 G1801130 G2902070 G2800670 G5300610 G3100190 G3701750 G2901010 G2200010 G5500490 G1300330 G3000670 G5500130 G2601270 G0600230 G2901860 G3900090 G5300090 G1702030 G2701290 G3600830 G1301110 G1700010 G

G5400170 G1200010 G4801550 G2000670 G0600410 G1200830 G2102090 G3701290 G1600810 G5100630 G2500170 G1900570 G5100610 G2201150 G0100010 G3800370 G4001210 G2001130 G3101710 G2601190 G2701030 G1700310 G1901350 G3900870 G0600150 G2001230 G4700770 G3000510 G3400050 G1900930 G0501350 G1901710 G2101090 G2101410 G1900630 G4803730 G4804090 G3600350 G2000170 G3701830 G5100330 G4803610 G1701770 G5400050 G2900990 G1701890 G4600890 G0101090 G4803510 G1200410 G4803630 G4600630 G3701670 G4700450 G1900970 G5501330 G4600730 G2300070 G4802810 G1300350 G4804410 G1800570 G2002050 G4001350 G2600490 G3800510 G2101050 G3700830 G3701550 G2102070 G5100750 G2902210 G1901570 G5500830 G3100390 G2601630 G3600130 G1901170 G2000030 G2800990 G4701710 G2200290 G5000070 G4700310 G4200490 G5500390 G0500710 G5500870 G3701350 G3800810 G2600510 G0800030 G4001370 G0101230 G4500530 G4804450 G4001030 G4701690 G3901530 G1700890 G4001290 G4802110 G1200590 G1700610 G1303070 G2001210 G2000530 G4801190 G0100090 G2500130 G3500490 G

G2101590 G1600090 G4701390 G1700510 G4601270 G2100050 G3400090 G1800670 G4200930 G5400790 G0201000 G3901490 G4700230 G3701050 G5101990 G1000030 G4000110 G2901870 G2900770 G1600750 G2000010 G4804190 G2600290 G3900750 G4701010 G3900610 G4804710 G2001750 G4200730 G3901330 G2101130 G1901150 G4700510 G1800450 G2901730 G1300230 G4802630 G1300130 G2900950 G4700270 G4801490 G1701210 G1801650 G3701070 G4500210 G2101610 G0501130 G1301710 G3700550 G2000790 G5000250 G2100470 G1303050 G1301170 G2101030 G1302630 G2102370 G2400150 G2102230 G1300570 G4200950 G2101750 G2601650 G1900230 G3600230 G1700710 G1800730 G1300730 G3100430 G3300170 G4200370 G2100990 G2901770 G1300250 G1900470 G5400290 G5101550 G2900010 G1801790 G3901010 G4500050 G2200550 G0900130 G2601550 G1700330 G2102170 G4801230 G1301330 G2801410 G2100910 G3701990 G5100970 G2102130 G5100850 G3200290 G0100610 G1302930 G2200890 G2000950 G2300230 G2700090 G1701390 G4800070 G3900330 G1800650 G4800850 G5400430 G3901070 G3900150 G5101270 G3700190 G

In [35]:
# pickle.dump(child_dfs, open('pickles/noaa/child_dfs.pkl', 'wb'))
child_dfs = pickle.load(open('pickles/noaa/child_dfs.pkl', 'rb'))

In [36]:
class TLModel:    
    def __init__(self, gis_join, time, model):
        self.rmse = rmse
        self.gis_join = gis_join
        self.time = time
        self.model = model
        
    def __str__(self):
        return f'{self.gis_join}: TLModel(time={self.time})'
    
class NonTLModel:
    def __init__(self, gis_join, time):
        self.gis_join = gis_join
        self.time = time
        
    def __str__(self):
        return f'{self.gis_join}: NonTLModel(time={self.time})'

# Child Models (Non-TL)

In [10]:
import time

non_tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    predict(child_dfs[gis_join])
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    non_tl_models[gis_join] = NonTLModel(gis_join, time2)
    counter += 1

Initial log joint probability = -2.03418
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7549.01   0.000559323       45988.9           1           1      124   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7973.35    0.00255003       23047.9           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       8009.92   0.000187361       13642.2           1           1      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       8107.64    0.00118167       9648.77           1           1      466   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       8199.37   0.000231663       7333.39           1           1      574   
    Iter      log prob        ||dx||      ||grad||       alpha  

NameError: name 'NonTLModel' is not defined

In [69]:
import dask

def predict_parallel(df_train, gis_join):
    time1 = time.monotonic()
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)
    time2 = time.monotonic()

    return gis_join, (time2 - time1)


gis_joins_list = []
child_dfs_list = []
cutoff = 10

for gis_join, df_ in child_dfs.items():
    gis_joins_list.append(gis_join)
    child_dfs_list.append(df_)


def train_one_child(df_, gis_join):
    gis_join, time_taken = predict_parallel(df_, gis_join)
    return gis_join, time_taken

assert len(gis_joins_list) == len(child_dfs_list)

lazy_results = []
for gis_join, df_ in zip(gis_joins_list[:cutoff], child_dfs_list[:cutoff]):
    lazy_result = dask.delayed(train_one_child)(df_, gis_join)
    lazy_results.append(lazy_result)

futures = dask.persist(*lazy_results)  # trigger computation in the background
results = dask.compute(*futures)
results[:5]

(('G3900930', 6.249335425905883),
 ('G0800870', 11.15669991611503),
 ('G5600290', 28.671380989952013),
 ('G4100370', 58.22061163187027),
 ('G0600070', 11.22736483020708))

In [71]:
len(results)

10

  ### Using Pandas UDFs and Spark

In [3]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
from pyspark import SparkContext
from pyspark.sql import SparkSession


jars = "" \
       "./jars/mongo-spark-connector_2.12-3.0.1.jar," \
       "./jars/mongo-java-driver-3.12.5.jar," \
       "./jars/bson-4.0.5.jar," \
       "./jars/spark-core_2.12-3.0.1.jar," \
       "./jars/spark-sql_2.12-3.0.1.jar"

SPARK_MASTER = os.environ["SPARK_MASTER"]
DB_NAME = os.environ["DB_NAME"]
DB_HOST = os.environ["DB_HOST"]
DB_PORT = os.environ["DB_PORT"]
# SPARK_THREAD_COUNT = os.environ["SPARK_THREAD_COUNT"]
SPARK_EXECUTOR_CORES = os.environ["SPARK_EXECUTOR_CORES"]
SPARK_EXECUTOR_MEMORY = os.environ["SPARK_EXECUTOR_MEMORY"]
SPARK_INITIAL_EXECUTORS = os.environ["SPARK_INITIAL_EXECUTORS"]
SPARK_MIN_EXECUTORS = os.environ["SPARK_MIN_EXECUTORS"]
SPARK_MAX_EXECUTORS = os.environ["SPARK_MAX_EXECUTORS"]
SPARK_BACKLOG_TIMEOUT = os.environ["SPARK_BACKLOG_TIMEOUT"]
SPARK_IDLE_TIMEOUT = os.environ["SPARK_IDLE_TIMEOUT"]

GISJOIN = "GISJOIN"

spark = SparkSession \
    .builder \
    .master(SPARK_MASTER) \
    .appName("NOAA Time-series - PySpark") \
    .config("spark.jars", jars) \
    .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
    .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true") \
    .config("park.executor.cores", SPARK_EXECUTOR_CORES) \
    .config("spark.executor.memory", SPARK_EXECUTOR_MEMORY) \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.initialExecutors", SPARK_INITIAL_EXECUTORS) \
    .config("spark.dynamicAllocation.minExecutors", SPARK_MIN_EXECUTORS) \
    .config("spark.dynamicAllocation.maxExecutors", SPARK_MAX_EXECUTORS) \
    .config("spark.dynamicAllocation.schedulerBacklogTimeout", SPARK_BACKLOG_TIMEOUT) \
    .config("spark.dynamicAllocation.executorIdleTimeout", SPARK_IDLE_TIMEOUT) \
    .getOrCreate()


result_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True)
])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def predict_udf(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return df_train_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

21/07/15 16:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/15 16:33:13 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
21/07/15 16:33:13 WARN ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


In [4]:
mongo_connection_uri = f'mongodb://{DB_HOST}:{DB_PORT}/{DB_NAME}.noaa_nam_2'

spark_df = spark.read.format("mongo").option("uri", mongo_connection_uri).load()

spark_df = spark_df.select('gis_join', 'year_month_day_hour', 'mean_sea_level_pressure_pascal')

spark_df.show()

results = (
    spark_df
    .groupBy('gis_join')
    .apply(predict_udf)
) 

/s/parsons/b/others/sustain/.local/lib/python3.8/site-packages/pyspark/sql/pandas/group_ops.py:73: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+--------+-------------------+------------------------------+
|gis_join|year_month_day_hour|mean_sea_level_pressure_pascal|
+--------+-------------------+------------------------------+
|G1200870|         2010011000|                      102111.0|
|G1200870|         2010011000|                      102199.0|
|G1200870|         2010011000|                      102197.0|
|G1200870|         2010011000|                      102207.0|
|G1200870|         2010011000|                      102133.0|
|G1200870|         2010011000|                      102214.0|
|G1200870|         2010011000|                      102229.0|
|G1200870|         2010011000|                      102246.0|
|G1200870|         2010011000|                      102219.0|
|G1200870|         2010011000|                      102216.0|
|G1200870|         2010011000|                      102239.0|
|G1200870|         2010011000|                      102258.0|
|G1200870|         2010011000|                      102244.0|
|G120087

# Child Models (TL)

In [199]:
def predict_transfer(df_train, trained_model):
    m_parent = trained_model
    # initilaize model with hyperparameters from parent model
    m = Prophet(
        seasonality_prior_scale = m_parent.seasonality_prior_scale,
        changepoint_prior_scale = m_parent.changepoint_prior_scale,
    )
    m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return m, df_train_future, df_train_forecast

tl_models = {}
counter = 1
for gis_join in list(child_dfs.keys())[:100]:
    time1 = time.monotonic()
    m, df_train_future, df_train_forecast = predict_transfer(child_dfs[gis_join], global_model)
    time2 = time.monotonic() - time1
    print(f'{counter} - {gis_join}: {time2} seconds')
    tl_models[gis_join] = TLModel(gis_join, time2, m)
    counter += 1 

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


Initial log joint probability = -2.03418
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7458.75     0.0366052       45531.3           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7668.24    0.00394961       17691.6           1           1      238   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       7849.71   0.000658998       32188.6           1           1      347   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       7990.92    0.00119234       9360.52           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       8025.72   9.10719e-05        4572.4      0.1594      0.1594      579   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


1 - G3900930: 4.039233192102984 seconds
Initial log joint probability = -2.13583
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       18798.8    0.00472404       76148.8           1           1      118   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       18962.5   0.000507454       33867.8           1           1      231   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       19044.1   0.000244076       13097.7           1           1      348   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       19126.4   0.000188566       7129.75           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       19165.7    0.00204759       16027.4           1           1      573   
    Iter      log prob  

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


...

97 - G2700410: 3.3173177039716393 seconds
Initial log joint probability = -2.19849
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       48847.4      0.021878        266575           1           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       49702.3    0.00485885        110114           1           1      241   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       49767.1   0.000671599       91335.9       0.476           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       49850.5   0.000194928         26235      0.4778      0.4778      471   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       49885.6   0.000608695       26212.2           1           1      582   
    Iter      log prob

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


98 - G3500590: 11.914384474046528 seconds
Initial log joint probability = -2.47046
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       86359.8     0.0372871        458900           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       87284.3    0.00335685        113374           1           1      242   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       87641.5    0.00196945        232072           1           1      357   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       87970.5    0.00225947        169878           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       88097.8   0.000365108       17949.1           1           1      576   
    Iter      log prob

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.


99 - G3500530: 24.94009071099572 seconds
Initial log joint probability = -2.11695
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       44051.3    0.00186064        157473           1           1      125   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       44756.8   0.000237376       30790.7      0.8289      0.8289      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       44988.2   0.000429668         18067           1           1      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       45641.9   9.95543e-05       29601.9           1           1      465   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       45925.5   0.000991869        114135           1           1      583   
    Iter      log prob 

## Collect Times taken to build Models

In [214]:
import pandas as pd

non_tl_times = [x.time for x in non_tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(non_tl_models.keys(), non_tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('non_tl_times.csv', index=False)

In [215]:
tl_times = [x.time for x in tl_models.values()]

df_non_tl = pd.DataFrame(list(zip(tl_models.keys(), tl_times)), columns=['GISJOIN', 'Time'])
df_non_tl.to_csv('tl_times.csv', index=False)